In [1]:
import os
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Корень проекта.
DIR_ROOT = Path.cwd().parent.parent
# Путь к удаленной директории с ресурсами: данные, модели и т.д.
DIR_REMOTE: Path | None = Path('/home/admin/cafa/resources')

if DIR_REMOTE is not None and DIR_REMOTE.exists():
    DIR_RESOURCE = DIR_REMOTE
else:
    DIR_RESOURCE = DIR_ROOT

In [ ]:
from parser import ia_parser

ia_dict = ia_parser(DIR_RESOURCE / 'data/raw/IA.txt')

In [7]:
from parser import obo_parser
from graph import Graph

obo_file = DIR_RESOURCE / 'data/raw/Train/go-basic.obo'

ontologies = []
for ns, terms_dict in obo_parser(obo_file).items():
    ontologies.append(Graph(ns, terms_dict, ia_dict, orphans=True))

In [12]:
# конвертировать файл tsv в txt, убрать столбец с корневыми онтологиями, чтобы привести к формату авторского решения

# train_terms = pd.read_csv(DIR_RESOURCE / 'data/raw/Train/train_terms.tsv', sep='\t', header=None)

# train_terms.drop(columns=[2], inplace=True)
# train_terms = train_terms.iloc[1:]

# train_terms.to_csv(DIR_RESOURCE / 'data/raw/Train/train_terms.txt', sep='\t',index=None)

In [8]:
from parser import gt_parser

gt_file = DIR_RESOURCE / 'data/raw/Train/train_terms.txt'

gt = gt_parser(gt_file, ontologies) # около 4Гб RAM

In [9]:
th_step = 0.001

tau_arr = np.arange(th_step, 1, th_step)

До этой строки потрачено 6,24 Гб Memory

In [10]:
from parser import pred_parser


file_name = DIR_RESOURCE / 'data/submission/submission_1000.txt'
prop = 'fill' # by default 'max', but author choosed fill
max_terms = 1500 # количество онтологий, которое мы выбрали при обучении


dfs = []

prediction = pred_parser(file_name, ontologies, gt, prop, max_terms)
df_pred = evaluate_prediction(prediction, gt, ontologies, tau_arr, args.norm, args.threads)
df_pred['filename'] = file_name
dfs.append(df_pred)

MemoryError: Unable to allocate 19.2 GiB for an array with shape (92210, 27942) and data type float64

In [ ]:
df = pd.concat(dfs)

# Save the dataframe
df = df[df['cov'] > 0].reset_index(drop=True)
df.set_index(['filename', 'ns', 'tau'], inplace=True)

if args.ia is not None:
    columns = ["cov", "pr", "rc", "f", "wcov", "wpr", "wrc", "wf", "mi", "ru", "s"]
else:
    columns = ["cov", "pr", "rc", "f"]
df.to_csv('{}/evaluation_all.tsv'.format(out_folder), columns=columns, float_format="%.5f", sep="\t")

# Calculate harmonic mean across namespaces for each evaluation metric
for metric, cols in [('f', ['rc', 'pr']), ('wf', ['wrc', 'wpr']), ('s', ['ru', 'mi'])]:
    if metric in columns:
        index_best = df.groupby(level=['filename', 'ns'])[metric].idxmax() if metric in ['f', 'wf'] else df.groupby(['filename', 'ns'])[metric].idxmin()

        df_best = df.loc[index_best]
        df_best['max_cov'] = df.reset_index('tau').loc[[ele[:-1] for ele in index_best]].groupby(level=['filename', 'ns'])['cov'].max()
        df_best.to_csv('{}/evaluation_best_{}.tsv'.format(out_folder, metric),
                        columns=columns + ["max_cov"], float_format="%.5f", sep="\t")